In [12]:
import pandas as pd
import gzip
import json
import re
import datetime
import numpy as np

In [52]:
data_source = "data/coinmarketcap_formatted.tsv"
all_data = pd.read_csv(data_source, sep='\t', header=0, encoding="utf-8", dtype={'Market Cap': np.float64, 'Total Market Cap': np.float64})
all_data["Market Cap Share"] = all_data["Market Cap"]/all_data["Total Market Cap"]

/opt/boxen/data/virturalenvs/p3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [87]:
def convert_coins_to_usd(date_data, coins):
    amount_usd = 0
    for coin_symbol, number_of_coins in coins.items():
        try:
            coin_usd = float(date_data[date_data["Symbol"] == coin_symbol]["Price"].values[0])
        except:
            coin_usd = 0
        amount_usd += number_of_coins * coin_usd
    return amount_usd

In [120]:
def buy_coins(date_data, amount_usd, buy_x_first, skip_x_first=0):
    max_pos = buy_x_first + skip_x_first
    coins_to_buy_data = date_data[(date_data["Pos"] <= max_pos) & (date_data["Pos"] > skip_x_first)]
    coins = {}
    coins_usd = {}
    for index, _ in coins_to_buy_data["Price"].iteritems():
        coin_usd = float(date_data["Price"][index])
        coin_symbol = date_data["Symbol"][index]
        normalize_market_cap_share = coins_to_buy_data["Market Cap Share"][index]/coins_to_buy_data["Market Cap Share"].sum().astype(np.float64)
        total_of_coin = 1.0 * normalize_market_cap_share * amount_usd / coin_usd
        coins[coin_symbol] = total_of_coin
        coins_usd[coin_symbol] = total_of_coin * coin_usd
    return coins, coins_usd

def buy_coins_equaly(date_data, amount_usd, buy_x_first, skip_x_first=0):
    max_pos = buy_x_first + skip_x_first
    coins_to_buy_data = date_data[(date_data["Pos"] <= max_pos) & (date_data["Pos"] > skip_x_first)]
    coins = {}
    coins_usd = {}
    for index, _ in coins_to_buy_data["Price"].iteritems():
        coin_usd = float(date_data["Price"][index])
        coin_symbol = date_data["Symbol"][index]
        total_of_coin = 1.0 * (amount_usd/buy_x_first) / coin_usd
        coins[coin_symbol] = total_of_coin
        coins_usd[coin_symbol] = total_of_coin * coin_usd
    return coins, coins_usd

def buy_coins_factor(date_data, amount_usd, buy_x_first, skip_x_first=0, factor=3):
    mult = factor
    step = (mult-1)/buy_x_first
    max_pos = buy_x_first + skip_x_first
    coins_to_buy_data = date_data[(date_data["Pos"] <= max_pos) & (date_data["Pos"] > skip_x_first)]
    coins = {}
    coins_usd = {}
    new_total_market_cap = 0
    curr_mult = 1
    for index, _ in coins_to_buy_data["Price"].iteritems():
        new_total_market_cap += curr_mult*coins_to_buy_data["Market Cap"][index]
        curr_mult += step

    curr_mult = 1
    for index, _ in coins_to_buy_data["Price"].iteritems():
        coin_usd = float(date_data["Price"][index])
        coin_symbol = date_data["Symbol"][index]
        normalize_market_cap_share = curr_mult*coins_to_buy_data["Market Cap"][index]/new_total_market_cap
        total_of_coin = 1.0 * normalize_market_cap_share * amount_usd / coin_usd
        coins[coin_symbol] = total_of_coin
        coins_usd[coin_symbol] = total_of_coin * coin_usd
        curr_mult += step
    return coins, coins_usd
    

In [89]:
def calculate_returns(buy_x_first, start_date, end_date, coins, skip_x_first=0):
    max_pos = buy_x_first + skip_x_first
    to_return = []
    last_coins = {}
    dates = sorted(list(all_data[(all_data["Date"] >= start_date) & (all_data["Date"] <= end_date)]["Date"].unique()))
    for date_ in dates:
        ## Convert to usd
        date_data = all_data[(all_data["Date"] == date_)]
        amount_usd = convert_coins_to_usd(date_data, coins)
        to_return.append([date_, round(amount_usd, 2)])

        # Buy top X coins
        coins_to_buy_data = date_data[(date_data["Pos"] <= max_pos) & (date_data["Pos"] > skip_x_first)]
        coins, coins_usd = buy_coins(date_data, amount_usd, buy_x_first, skip_x_first=skip_x_first)
        last_coins = [coins, coins_usd]
        
    return to_return, last_coins


In [63]:
#You have to select a date that is available in the dataset
def print_returns_usings_topx(start_date, end_date, initial_coins, buy_x_firsts, show_coins_to_buy=False, skip_x_first=0):
    print("From: %s to %s" %(start_date, end_date))
    initial = True
    coins_to_buy = {}
    print("Top X Coins\tFinal Growth")
    for buy_x_first in buy_x_firsts:
        calculated_values, coins_to_buy = calculate_returns(buy_x_first, start_date, end_date, initial_coins, skip_x_first=skip_x_first)
        print("%s\t%s" % (buy_x_first, "%s%%" % round(100* (calculated_values[-1][1]/calculated_values[0][1]-1),2)))
    if show_coins_to_buy:
        print("")
        print("\t".join(["Coin", "Total Coins", "Coin Price USD", "Total USD"]))
        for coin in coins_to_buy[0].keys():
            print(
                "\t".join([
                    str(coin),
                    str(coins_to_buy[0][coin]),
                    str(coins_to_buy[1][coin]/coins_to_buy[0][coin]), 
                    str(coins_to_buy[1][coin])
                ])
            )


In [69]:
start_date, end_date = "2017-01-01", "2017-04-16"
initial_coins = {
    "BTC": 10
}
buy_x_firsts = [1,2,3,5,10,20,100]
skip_x_first = 2
print_returns_usings_topx(start_date, end_date, initial_coins, buy_x_firsts, skip_x_first=skip_x_first)


From: 2017-01-01 to 2017-04-16
Top X Coins	Final Growth
1	-100.0%
2	-100.0%
3	112.27%
5	232.66%
10	228.6%
20	200.17%
100	198.93%


In [59]:
start_date, end_date = "2017-01-01", "2017-04-16"
initial_coins = {
    "BTC": 10
}
buy_x_firsts = [1,2,3,5,10,20,100]
skip_x_first = 0
print_returns_usings_topx(start_date, end_date, initial_coins, buy_x_firsts)


From: 2017-01-01 to 2017-04-16
Top X Coins	Final Growth
1	22.43%
2	43.59%
3	45.79%
5	51.35%
10	54.3%
20	54.91%
100	57.5%


In [8]:
start_date, end_date = "2017-04-23", "2017-04-23"
initial_coins = {
    "BTC": 45
}
buy_x_firsts = [5]
print_returns_usings_topx(start_date, end_date, initial_coins, buy_x_firsts, show_coins_to_buy=True)



From: 2017-04-23 to 2017-04-23
Top X Coins	Final Growth
5	0.0%

Coin	Total Coins	Coin Price USD	Total USD
BTC	33.5320383522	1224.68	41066.0167292
ETH	187.265866776	48.47	9076.77656265
XRP	77988.1300197	0.031566	2461.7733122
LTC	104.523714191	13.85	1447.65344155
DASH	14.9193678375	70.94	1058.37995439


In [134]:
start_date = "2016-04-17"
amount_usd = 10000
buy_x_first = 5
skip_x_first = 0
date_data = all_data[(all_data["Date"] == start_date)]
coins, coins_usd = buy_coins(date_data, amount_usd, buy_x_first, skip_x_first)
coins_eq, coins_eq_usd = buy_coins_equaly(date_data, amount_usd, buy_x_first, skip_x_first)
coins_factor, coins_factor_usd = buy_coins_factor(date_data, amount_usd, buy_x_first, skip_x_first, factor=2)

end_date = "2017-04-23"
date_data = all_data[(all_data["Date"] == end_date)]
print(convert_coins_to_usd(date_data, coins))
print(convert_coins_to_usd(date_data, coins_eq))
print(convert_coins_to_usd(date_data, coins_factor))

32077.6854151
56994.74096037766
33069.9491092
